In [1]:
import numpy as np
import scipy.special as spsp
import scipy.linalg as spla

import sourcedefender
from HomeworkFramework import Function

class RS_optimizer(Function): # need to inherit this class "Function"
    def __init__(self, target_func):
        super().__init__(target_func) # must have this init to work normally

        self.lower = self.f.lower(target_func)
        self.upper = self.f.upper(target_func)
        self.dim = self.f.dimension(target_func)

        self.target_func = target_func

        self.eval_times = 0
        self.optimal_value = float("inf")
        self.optimal_solution = np.empty(self.dim)
        
        self.step_size = 0.5
        self.mean_vec = np.ones(self.dim)
        self.Cov = np.identity(len(self.mean_vec))
        self.Pc = 0
        self.Ps = 0
        self.Cc = 4/self.dim
        self.Cs = 4/self.dim
        self.C1 = 2/(self.dim ** 2)
        self.lamba = int(4 + np.floor(3 * np.log(self.dim)))
        self.Mu = int(self.lamba/2)
        self.weights = np.ones(self.Mu)/self.Mu
        self.Muw = 1/sum(self.weights**2)
        self.Cmu = self.Muw/(self.dim ** 2)
        self.Ds = 1 + np.sqrt(self.Mu/self.dim)


    def get_optimal(self):
        return self.optimal_solution, self.optimal_value

    def run(self, FES): # main part for your implementation
        while self.eval_times < FES:
            print('=====================FE=====================')
            print(self.eval_times)
            
            y = np.random.multivariate_normal(np.zeros(self.dim), self.Cov, size=self.lamba)
            x = self.mean_vec + self.step_size * y
            ReachFunctionLimit = False
            x[x > self.upper] = self.upper
            x[x < self.lower] = self.lower
            val = []
            for i in range(len(x)):
                v = self.f.evaluate(func_num, x[i])
                val.append(v)
                self.eval_times += 1
            con_mat = np.c_[val, x, y]
            con_mat = con_mat[con_mat[:, 0].argsort()]
            y = con_mat[:, (self.dim + 1):]
            x = con_mat[:, 1:(self.dim + 1)]
            value = con_mat[0, 0]
            solution = x[0, :]
            
            Yw = 0
            for i in range(self.Mu):
                Yw += self.weights[i] * y[i]
            self.mean_vec = self.mean_vec + self.step_size * Yw
            if np.linalg.norm(self.Ps) < (1.5*np.sqrt(self.dim)):
                self.Pc = ((1 - self.Cc) * self.Pc + np.sqrt(1 - (1-self.Cc) ** 2) * np.sqrt(self.Muw) * Yw)
            else:
                self.Pc = (1 - self.Cc) * self.Pc
            
            C_12 = spla.inv(spla.sqrtm(self.Cov))
            self.Ps = (1 - self.Cs) * self.Ps + np.sqrt(1 - (1-self.Cs) ** 2) * np.sqrt(self.Muw) * spla.solve_triangular(C_12, Yw, lower=True)
            Cm = (self.weights[:self.Mu] * np.transpose(y[:self.Mu])).dot(y[:self.Mu])
            self.Cov = (1 - self.C1 - self.Cmu) * self.Cov + self.C1 * self.Pc * np.transpose(self.Pc) + self.Cmu * Cm
            Ps_nor = np.linalg.norm(self.Ps)
            E_nor = (np.sqrt(2) * spsp.gamma((self.dim + 1) / 2) / spsp.gamma(self.dim / 2))
            self.step_size = self.step_size * np.exp(self.Cs / self.Ds * (Ps_nor / E_nor - 1))
            

#             solution = np.random.uniform(np.full(self.dim, self.lower), np.full(self.dim, self.upper), self.dim)
#             value = self.f.evaluate(func_num, solution)
#             self.eval_times += 1

            if value == "ReachFunctionLimit":
                print("ReachFunctionLimit")
                break            
            if float(value) < self.optimal_value:
                self.optimal_solution[:] = solution
                self.optimal_value = float(value)

            print("optimal: %f\n" % self.get_optimal()[1])
            

if __name__ == '__main__':
    func_num = 1
    fes = 0
    #function1: 1000, function2: 1500, function3: 2000, function4: 2500
    while func_num < 5:
        if func_num == 1:
            fes = 1000
        elif func_num == 2:
            fes = 1500
        elif func_num == 3:
            fes = 2000 
        else:
            fes = 2500

        # you should implement your optimizer
        op = RS_optimizer(func_num)
        op.run(fes)
        
        best_input, best_value = op.get_optimal()
        print(best_input, best_value)
        
        # change the name of this file to your student_ID and it will output properlly
        with open("{}_function{}.txt".format(__file__.split('_')[0], func_num), 'w+') as f:
            for i in range(op.dim):
                f.write("%f\n" % best_input[i])
            f.write("%f\n" % best_value)
        func_num += 1 


=====================FE=====================
0
optimal: 2.706223

=====================FE=====================
9
optimal: 1.927339

=====================FE=====================
18
optimal: 1.756237

=====================FE=====================
27
optimal: 1.756237

=====================FE=====================
36
optimal: 0.756010

=====================FE=====================
45
optimal: 0.756010

=====================FE=====================
54
optimal: 0.756010

=====================FE=====================
63
optimal: 0.756010

=====================FE=====================
72
optimal: 0.756010

=====================FE=====================
81
optimal: 0.756010

=====================FE=====================
90
optimal: 0.756010

=====================FE=====================
99
optimal: 0.756010

=====================FE=====================
108
optimal: 0.756010

=====================FE=====================
117
optimal: 0.756010

=====================FE=====================
126
optimal: 0.75

/Users/kuohuichi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: covariance is not symmetric positive-semidefinite.


TypeError: ufunc 'multiply' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')